These are my notes as I follow along the [first lecture of TensorFlow for Research.](https://www.youtube.com/watch?v=g-EvyKpZjmQ&list=PLQ0sVbIj3URf94DQtGPJV629ctn2c1zN-&index=1)

First, I will have a code snippet to be able to run tensorboard on Google colab. If you are running this locally, you wont need to do this

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-07-20 15:59:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.230.122, 52.22.145.207, 52.73.9.93, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.230.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.4’

ngrok-stable-linux- 100%[===================>]  12.98M  13.4MB/s    in 1.0s    

2019-07-20 15:59:06 (13.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.4’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
https://fe08302a.ngrok.io


In [0]:
import tensorflow as tf

In [10]:
a = tf.add(2, 3)
b = tf.add(3.0, 5.0)
print(a)
print(b)

Tensor("Add_2:0", shape=(), dtype=int32)
Tensor("Add_3:0", shape=(), dtype=float32)


The computation is not run yet, all we've done is defined the graph structure. As you can see in dtype, the nodes can infer their data types from their inputs. How do we get the value of a and b? With a **session**.

A session gets all the resources in place (CPUs, GPUs ) to actually run the computation.

In [14]:
sess = tf.Session()
print(sess.run(a))
sess.close()


5


What TF does here is to look at the "a" and will compute everything that it needs to get the value of a. As you can see, it doesn't need to compute the value of b.



In [18]:
with tf.Session() as sess:
  print(sess.run(a))

5


The above is equivalent to what we did before (sess = tf....)

In [0]:
x = 2
y = 3
op1 = tf.add(x, y)
op2 = tf.multiply(x, y)
op3 = tf.pow(op1, op2)

with tf.Session() as sess:
  op3 = sess.run(op3)

Now TF will need to calculate op3. What does op3 depend on? op1 and op2, so now TF wil have to calculate everything we defined

In [23]:
op3, (x + y) ** (x * y)

(7776, 15625)

You can pass a whole list of nodes you cant to compute and tensorflow will return a list of the results

In [25]:
x = 2
y = 3
op1 = tf.add(x, y)
op2 = tf.multiply(x, y)
useless = tf.multiply(x, op1)
op3 = tf.pow(op1, op2)

with tf.Session() as sess:
  op3_run, useless_run = sess.run([op3, useless])

print( op3_run, useless_run)

15625 10


The great thing about having subgraphs that we can explicitly run is that we can parallelize our code inmensely by telling TF to run subgraph 1 on the GPU 0, subgraph 1 on GPU 1, and so on. But we will see this with more detail a few weeks from now.

# What if you want more graphs?
Try not to, lol. Just have disconnected subgraphs within one default graph.

In [27]:
g = tf.Graph()

with g.as_default():
   x = tf.add(3, 5)
    
sess = tf.Session(graph=g)
x_result = sess.run(x)
sess.close()
print(x_result)

8


Here we see how we would explicitly handle graphs, but usually we won't need to do this. But its brittle because you may do something like this:

In [0]:
g = tf.Graph()

#This will add an op to the DEFAULT graph, not g
a = tf.constant(3)

#This, instead, will do for g
with g.as_default():
  b = tf.constant(5)


This is very prone to errors because you don't know which nodes belong to each graph and you can't mix and match. I you really want to do it, do it more explicitly but even better don't do it!it's not to do it!

# Why graphs? 


*   Because it gives you explicit control on what gets ran.
*   Breaks up computation into small, self contained units (more important for implementation than for the user).
*   Makes it easier for distributed computation.
*   Many common ML algorithms are usually represented as directed graphs, so it's easy to translate them into tensorflow.



# The End